**Import libraries**

In [1]:
from glob import glob 
from google.colab import files
import matplotlib.image as mpimg
import pickle
import random
import matplotlib.pyplot as plt
import csv
import numpy as np
import cv2
import progressbar
import datetime

**Retrieve images from external sources**

In [2]:
# Clone the GitHub repo of mahjong tile images
!git clone https://github.com/camerash/mahjong-dataset

Cloning into 'mahjong-dataset'...
remote: Enumerating objects: 2565, done.
remote: Total 2565 (delta 0), reused 0 (delta 0), pack-reused 2565
Receiving objects: 100% (2565/2565), 59.48 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [3]:
# Download Describable Textures Dataset (DTD)
!wget https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz

--2021-10-04 23:39:15--  https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625239812 (596M) [application/x-gzip]
Saving to: ‘dtd-r1.0.1.tar.gz’

dtd-r1.0.1.tar.gz   100%[===================>] 596.27M  21.4MB/s    in 29s     

2021-10-04 23:39:45 (20.3 MB/s) - ‘dtd-r1.0.1.tar.gz’ saved [625239812/625239812]



In [4]:
# Extract the DTD
!tar xf dtd-r1.0.1.tar.gz

In [5]:
# Delete the zip folder of DTD
!rm dtd-r1.0.1.tar.gz

In [ ]:
# Load all *jpg from dtd subdirectories and save them in a pickle file (1x)

backgrounds_pck_fn="backgrounds.pck"
dtd_dir="dtd/images/"
bg_images=[]
print("Loading images ... (It could take several minutes)")
for subdir in glob(dtd_dir+"/*"):
    for f in glob(subdir+"/*.jpg"):
        bg_images.append(mpimg.imread(f))
pickle.dump(bg_images,open(backgrounds_pck_fn,'wb'))

class Backgrounds():
    def __init__(self,backgrounds_pck_fn=backgrounds_pck_fn):
        self._images=pickle.load(open(backgrounds_pck_fn,'rb'))
        self._nb_images=len(self._images)
        print("Number of images loaded :", self._nb_images)
    def get_random(self, display=False):
        bg=self._images[random.randint(0,self._nb_images-1)]
        if display: plt.imshow(bg)
        return bg

backgrounds = Backgrounds()

**Edit directory**

In [7]:
# Create a directory that will contain the images we generate
data_dir="images"
!mkdir images

**Define functions**

In [8]:
# Rotate the image to the angle as specified
def rotate_tile(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    border_color=(255, 255, 255)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e. the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

In [9]:
# Resize an image
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [10]:
def create_image_in_grid_format(csv_filename, grid_length=4):
  (tile_images, tile_type_of_tile_images) = genrate_images(grid_length)
  tile_type_of_tile_images_for_output = tile_type_of_tile_images[:]

  # Randomly select a background image
  background_image = backgrounds.get_random()

  # Copy the background image such that the image would not be mutated  
  output_image = cv2.cvtColor(background_image, cv2.COLOR_RGB2RGBA).copy()
  
  # Resize the background image to standard width and height (i.e. 1024x1024)
  # The resize action would result int distortion
  resize_specifications = (image_standard_width, image_standard_height)
  output_image = cv2.resize(output_image, resize_specifications, 
                            interpolation = cv2.INTER_AREA)
  csv_row_data_placeholder = []

  # Place the grid on the resized background image, 
  # where the black area of the grid would be perceived as transparent
  for row_index in range(grid_length): 
    for column_index in range(grid_length):
      tile_img = tile_images[0]
      tmp = cv2.cvtColor(tile_img, cv2.COLOR_BGR2GRAY)
      _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
      b, g, r, a = cv2.split(tile_img)
      rgba = [b,g,r, alpha]
      tile_img = cv2.merge(rgba,4)

      x_offset = int(((column_index / grid_length) * output_image.shape[1]))
      y_offset = int((((row_index) / grid_length) * output_image.shape[0]))

      y1, y2 = y_offset, y_offset + tile_img.shape[0]
      x1, x2 = x_offset, x_offset + tile_img.shape[1]
      
      alpha_s = tile_img[:, :, 3] / 255.0
      alpha_l = 1.0 - alpha_s
      
      for c in range(0, 3):
        output_image[y1:y2, x1:x2, c] = (alpha_s * tile_img[:, :, c] + alpha_l 
                                         * output_image[y1:y2, x1:x2, c])
        
      # Compute the value of relative coordinates
      min_x = x1/output_image.shape[1]
      max_x = x2/output_image.shape[1]
      min_y = y1/output_image.shape[0]
      max_y = y2/output_image.shape[0]

      # Write data to csv: 
      # (1) the image filename, 
      # (2) tile type and 
      # (3) top-left and bottom-right vertices coordinates 
      #     of the mahjong tile in the image 
      csv_row_data_placeholder.append(
          (tile_type_of_tile_images[0],min_x,min_y,max_x,max_y))
      
      del tile_images[0]
      del tile_type_of_tile_images[0]

  # Save the image
  now = datetime.datetime.now()
  
  string_of_current_moment = str(now.year) + '-' + \
                              str(now.month).zfill(2) + '-' + \
                              str(now.day).zfill(2) + '-'  + \
                              str(now.hour).zfill(2) + '-' + \
                              str(now.minute).zfill(2) + '-' + \
                              str(now.second).zfill(2) + '-' + \
                              str(now.microsecond)  
  
  filename_of_img_created = data_dir + '/' + string_of_current_moment + '.jpg'
  cv2.imwrite(filename_of_img_created, output_image)

  # Append row in csv to record labelling information
  dataset_types = ['TRAIN', 'TEST', 'VALIDATION']
  set_type = random.choices(dataset_types, weights=(80,10,10), k=1)[0]
  
  for row_data in csv_row_data_placeholder:
    (target_tile_type,min_x,min_y,max_x,max_y) = row_data
    row_data_to_be_written_in_csv = [set_type,
                                     filename_of_img_created,
                                     target_tile_type,
                                     min_x,min_y,'','',
                                     max_x,max_y,'',''] 
    append_row_to_csv(csv_filename,row_data_to_be_written_in_csv, 'a')

  return tile_type_of_tile_images_for_output

In [11]:
def genrate_images(grid_length=4):  

  directory_of_github_repo_sampling_images = 'mahjong-dataset/tiles-resized/'

  grid_size = grid_length * grid_length

  tile_images = []
  tile_type_of_tile_images = []

  for grid_index in range(grid_size):    
    selected_tile_type = random.choice(list(lookup_filename_of_tile_img.keys()))
    image_samples = lookup_filename_of_tile_img[selected_tile_type]
    selected_sample_image = cv2.imread(
        directory_of_github_repo_sampling_images + \
        random.choices(image_samples, k=1)[0])
    
    selected_sample_image = cv2.cvtColor(selected_sample_image, 
                                         cv2.COLOR_RGB2RGBA).copy()  
    
    # Rotate the image
    rotated_sample_image = rotate_tile(selected_sample_image, 
                                       random.randrange(5,355))

    # The size of the image would change after rotation, 
    # and thus the rotated image has to be resized, 
    # by specifying its width to the standard width
    resized_sample_image = \
    image_resize(rotated_sample_image, 
                 height=int(image_standard_height/(grid_length * 1.5)))
    resized_sample_image = \
    image_resize(resized_sample_image, 
                 width=int(image_standard_width/(grid_length * 1.5)))
    tile_images.append(resized_sample_image)
    tile_type_of_tile_images.append(selected_tile_type)

  return (tile_images, tile_type_of_tile_images)

In [12]:
# Check if the annotation target is met.  Annotation target is met only when 
# all tile types have the annotation number that is 
# equal or larger than the threshold
def check_if_annotation_target_is_met(tile_type_annotations, 
                                      annotations_target_threshold):
  for annotation_occurrence in tile_type_annotations.values():
    if annotation_occurrence < annotations_target_threshold:
      return False
  
  return True

In [13]:
# Decalre variables to set the size of images in the dataset
image_standard_width = image_standard_height = 1024

# Declare variables to determine the size of grid.
# E.g. for a 4x4 grid, in each image there would be 16 mahjong tiles.
min_grid_length = 2
max_grid_length = 8

# Declare the variable to determine 
# how many annotations are needed for each tile type
target_number_of_annotations_for_each_tile_types = 150

In [14]:
# Count the annotation number for determining the progress % in the progress bar
def count_annotations_number_for_updating_progress_bar(tile_type_annotations, 
                                                      annotations_target):
    result = 0
    for annotation_occurrence in tile_type_annotations.values():
      if annotation_occurrence < annotations_target:
        result += annotation_occurrence
      else:
        result += annotations_target
    return result

In [15]:
# Write data to a csv file
def append_row_to_csv(csv_filename, data, write_mode):
  with open(csv_filename, write_mode, encoding='UTF8') \
  as dataset_csvfile_for_model_training:
    writer = csv.writer(dataset_csvfile_for_model_training)
    writer.writerow(data)
    dataset_csvfile_for_model_training.close()

In [16]:
# Read labelling data from the csv file

lookup_filename_of_tile_img = {}
tile_type_annotations = {}

with open('mahjong-dataset/tiles-data/data.csv', newline='') as csvfile:
  table = csv.reader(csvfile, delimiter=' ')
  for row in table:
    row_data = row[0].split(',')
    img_file = row_data[0]
    tile_type = row_data[2]
    if (tile_type == 'label-name' or ('bonus-' in tile_type)):
      continue
    if ((tile_type in tile_type_annotations) is False):
      tile_type_annotations[tile_type] = 0

    if (tile_type in lookup_filename_of_tile_img):
      lookup_filename_of_tile_img[tile_type].append(img_file)
    else:
      lookup_filename_of_tile_img[tile_type] = [img_file]
  csvfile.close()

In [17]:
# Create a csv file that would contain the annotation information of the dataset images
csv_filename = 'dataset-for-training-AutoML-Vision-model.csv'
header = ['set','path','label','x_min','y_min','x_max',
          'y_min','x_max','y_max','x_min','y_max']
append_row_to_csv(csv_filename, header, 'w')

# Create a progress bar to display progress
total_number_of_tile_types = 34
progress_bar_max_val = total_number_of_tile_types * \
                        target_number_of_annotations_for_each_tile_types

bar = progressbar.ProgressBar(max_value=progress_bar_max_val, 
                              min_value=0, 
                              widgets=[progressbar.Bar('=', '[', ']'),
                                       ' ', 
                                       progressbar.Percentage()])
annotations_number_for_updaing_progress_bar = 0
bar.start()
bar.update(annotations_number_for_updaing_progress_bar)

# Generate images until the number of annotation target is met
annotation_target_is_met = False
total_number_of_images_generated = 0
while (annotation_target_is_met is False):  
  grid_length = random.randint(min_grid_length, max_grid_length)
  tile_types_annotated = create_image_in_grid_format(csv_filename, 
                                                     grid_length)
  total_number_of_images_generated += 1
  for tile_type in tile_types_annotated:
    tile_type_annotations[tile_type] += 1
  annotations_number_for_updaing_progress_bar = \
  count_annotations_number_for_updaing_progress_bar(
      tile_type_annotations, 
      target_number_of_annotations_for_each_tile_types)
  bar.update(annotations_number_for_updaing_progress_bar)
  annotation_target_is_met = \
  check_if_annotation_target_is_met(tile_type_annotations, target_number_of_annotations_for_each_tile_types)

bar.finish()
print("Total number of images generated: " + str(total_number_of_images_generated))

total_number_of_annotations = sum(tile_type_annotations.values())
print("Total number of annotations created: " + str(total_number_of_annotations))

[========================================================================] 100%


Total number of images generated: 193
Total number of annotations created: 6011
